# CPUs plot with Altair/Vega

Plots
- by name
- by launch date

work in progress to:
- show side by side the CPU perf by name sorted by perf and the plot by launch date
  - with highlighting selection
- change the sorting order interactively. See https://stackoverflow.com/questions/67379937/change-mark-order-via-parameter (doesn't work yet...) and https://altair-viz.github.io/user_guide/transform/window.html

PH, Feb 2025

In [1]:
import pandas as pd
import altair as alt

load CSV export of Baserow table (CSV exported in [Retrieve_baserow.ipynb](Retrieve_baserow.ipynb))

In [2]:
cpus = pd.read_csv('CPUs.csv', parse_dates=['Launch date'])
cpus

,Name,Launch date,Cores,Age,Architecture,GB6 Single,GB6 Multi,Designer,Win11
0,Intel Core i5-6200U,2015-07-01,2,9.7,Skylake,916.0,1832.0,Intel,No
1,Intel Core i5-6300U,2015-07-01,2,9.7,Skylake,965.0,1910.0,Intel,No
2,Intel Core i5-7300U,2017-01-01,2,8.1,Kaby Lake,1088.0,2030.0,Intel,No
3,Intel Core i5-8250U,2017-07-01,4,7.6,Kaby Lake R,1145.0,3126.0,Intel,Yes
4,Intel Core i5-8350U,2017-07-01,4,7.6,Kaby Lake R,1191.0,3237.0,Intel,Yes
5,Intel Core i5-8365U,2019-04-01,4,5.9,Whiskey Lake,1270.0,3167.0,Intel,Yes
6,Intel Core i5-9400H,2019-04-01,4,5.9,Coffee Lake,1429.0,4347.0,Intel,Yes
7,Intel Core i5-10210U,2019-07-01,4,5.7,Comet Lake,1229.0,3088.0,Intel,Yes
8,Intel Core i5-10310U,2020-04-01,4,4.9,Comet Lake,1297.0,3450.0,Intel,Yes
9,Intel Core i5-1135G7,2020-07-01,4,4.6,Tiger Lake,1702.0,4694.0,Intel,Yes


In [3]:
cpu_list_plot = [
    'Intel Core i5-6200U',
    'Intel Core i5-6300U',
    'Intel Core i5-7300U',
    'Intel Core i5-8250U',
    'Intel Core i5-8350U',
    'Intel Core i5-8365U',
    #'Intel Core i5-9400H', # not a U-series processor
    'Intel Core i5-10210U',
    'Intel Core i5-10310U',
    'Intel Core i5-1135G7',
    'Intel Core i5-1145G7',
    'Intel Core i5-1245U',
    'Intel Core i5-1335U',
    'AMD Ryzen 7 PRO 2700U',
    'AMD Ryzen 5 PRO 3500U',
    'AMD Ryzen 5 PRO 4650U',
    'AMD Ryzen 5 PRO 5650U',
]

In [4]:
i_drop = []
for i in range(len(cpus)):
    if cpus.iloc[i].Name not in cpu_list_plot:
        i_drop.append(i)
print("dropped rows: ", i_drop)
cpus.drop(i_drop, inplace=True)

dropped rows:  [6]


## Plot

### By launch date

In [34]:
highlight = alt.selection_point(name="highlight", on="pointerover", empty=False)
size = alt.when(highlight).then(alt.value(300))
color_w11 = alt.when(highlight).then(alt.value('red')).otherwise('Win11')
color_w11 = alt.when(highlight, empty=True).then('Win11').otherwise(alt.value('lightgray'))
#order = alt.when(highlight).then(alt.value(1)).otherwise(alt.value(0))

In [35]:
chart_date = alt.Chart(cpus).mark_point(size=150, filled=True, strokeWidth=5).encode(
    y='Launch date',
    tooltip='Name',
    color=color_w11,#'Win11',
    #fillOpacity='Win11',
    shape='Designer',
    #size=size,
).add_params(
    highlight
)

chart_dsingle = chart_date.encode(
    x='GB6 Single',
).properties(
    title='Single-Core Performance'
)
chart_dmulti = chart_date.encode(
    x='GB6 Multi',
    y=alt.Y('Launch date', axis=alt.Axis(labels=False, title='')) # hide y tick labels
).properties(
    title='Multi-Core Performance'
)

(chart_dsingle | chart_dmulti).configure_range(
    category={'scheme': 'set1'}
)

alt.HConcatChart(...)

### CPUs sorter by perf

In [59]:
color_age = alt.when(highlight).then(alt.value('red')).otherwise('Age:Q')

In [61]:
chart_name = alt.Chart(cpus).mark_point(size=100, filled=True).encode(
    alt.Y('Name').sort(field='GB6 Single', order='descending'),
    tooltip='Name',
    color=color_age,#'Age',
    shape='Designer',
    size=size
).add_params(
    highlight
)

chart_nsingle = chart_name.encode(
    x='GB6 Single',
).properties(
    title='Single-Core Performance'
)

chart_nmulti = chart_name.encode(
    x='GB6 Multi',
    y=alt.Y('Name', axis=alt.Axis(labels=False, title='')).sort(field='GB6 Single', order='descending') # hide y tick labels
).properties(
    title='Multi-Core Performance'
)
chart_nsingle | chart_nmulti

alt.HConcatChart(...)

In [62]:
(chart_nsingle | chart_nmulti) & \
(chart_dsingle | chart_dmulti)

alt.VConcatChart(...)

---
Attempt to make the ordering changebable

In [11]:
index_select = alt.binding_select(options=['GB6 Single', 'GB6 Multi'], name='sort field:')
index_param = alt.param(bind=index_select)
index_param

Parameter('param_1', VariableParameter({
  bind: BindRadioSelect({
    input: 'select',
    name: 'sort field:',
    options: ['GB6 Single', 'GB6 Multi']
  }),
  name: 'param_1'
}))

Note: adding the param as sort field doesn't work. Read example more carefully? https://altair-viz.github.io/gallery/multiple_interactions.html#gallery-multiple-interactions

In [51]:
chart_name = alt.Chart(cpus).transform_window(
    sort=[{'field': 'param_2'}],
    frame=[None, 0],
    perf_sorted='rank(*)'
).mark_point(size=100, filled=True).encode(
    alt.Y('Name').sort(field='perf_sorted', order='descending'),#.sort(field='GB6 Single', order='descending'),
    tooltip='Name',
    #color='Win11',
    color='Age',
    shape='Designer'
).add_params(index_param)

chart_nsingle = chart_name.encode(
    x='GB6 Single',
).properties(
    title='Single-Core Performance'
)
chart_nsingle

alt.Chart(...)